In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [134]:
vcap = cv2.VideoCapture('mario_1.webm') # 0=camera
vcopy = vcap

In [135]:
if vcopy.isOpened():
    width = vcopy.get(3)
    height = vcopy.get(4)
#     frame = frame[240:700, 250:620]
    print(width, height)

854.0 480.0


In [132]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [136]:
while True:
    if vcopy.grab():
        flag, frame = vcopy.retrieve()
#         frame = frame[240:700, 250:620] ## GoT frame part
        frame = frame[50:230, 300:550] ## Human Face - when 3 ppl are present
#         frame = frame[30:200, 120:300]
#         frame = frame[0:100, 0:200]
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray,
                                        scaleFactor=1.3,
                                        minNeighbors=3,
                                        minSize=(30, 30))
        if len(faces) > 0:
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
                frame = frame[y:y + h, x:x + w]

#         tight = cv2.Canny(frame, 400, 470)z
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         contours, hierarchy = cv2.findContours(tight, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#         cframe = cv2.drawContours(frame, contours, -1, (0,255,0), 3)
        
        if not flag:
            continue
        else:
            cv2.imshow('video', frame) 
    if cv2.waitKey(33) == ord('a'):
        break